## Anexo I - Programa responsável por gerar e armazenar recomendações de filmes. Baseado na técnica de filtro colaborativo e algoritmo de vizinhos mais próximos.


In [11]:
import pandas as pd
from pymongo import MongoClient

In [12]:
dataFile='ratings.csv'
data=pd.read_csv(dataFile,sep=",",header=0,names=["userId","movieId","rating","timestamp"])

In [13]:
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [14]:
movieFile='movies.csv'
movies=pd.read_csv(movieFile,sep=",",header=0,names=["movieId","title","genres"])

In [15]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
def movieMeta(movieId):
    title = movies.at[movieId,"title"]
    genre = movies.at[movieId,"genres"]
    return title, genre

movieMeta(2)

('Grumpier Old Men (1995)', 'Comedy|Romance')

In [17]:
data = data[data["movieId"].isin(movies.index)]

In [18]:
def faveMovies(user,N):
    userRatings = data[data["userId"]==user]
    sortedRatings = pd.DataFrame.sort_values(userRatings,['rating'],ascending=[0])[:N] 
    sortedRatings["title"] = sortedRatings["movieId"].apply(movieMeta)
    return sortedRatings

In [19]:
faveMovies(1,10)

,userId,movieId,rating,timestamp,title
4,1,1172,4.0,1260759205,"(Gridlock'd (1997), Crime)"
13,1,2105,4.0,1260759139,"(Besieged (a.k.a. L' Assedio) (1998), Drama)"
12,1,1953,4.0,1260759191,"(At First Sight (1999), Drama)"
8,1,1339,3.5,1260759125,"(Red Corner (1997), Crime|Thriller)"
19,1,3671,3.0,1260759117,"(War of the Roses, The (1989), Comedy|Drama)"
1,1,1029,3.0,1260759179,"(Highlander (1986), Action|Adventure|Fantasy)"
2,1,1061,3.0,1260759182,"(Hype! (1996), Documentary)"
14,1,2150,3.0,1260759194,"(General's Daughter, The (1999), Crime|Drama|M..."
17,1,2455,2.5,1260759113,"(Pokémon: The First Movie (1998), Adventure|An..."
0,1,31,2.5,1260759144,"(Twelve Monkeys (a.k.a. 12 Monkeys) (1995), My..."


In [20]:
data.shape

(84739, 4)

In [21]:
usersPerMOVIE = data.movieId.value_counts()
usersPerMOVIE.head(10)

356     341
296     324
318     311
593     304
260     291
480     274
2571    259
1       247
527     244
589     237
Name: movieId, dtype: int64

In [22]:
usersPerMOVIE.shape

(5642,)

In [23]:
MOVIESsPerUser = data.userId.value_counts()

In [24]:
MOVIESsPerUser.shape

(671,)

In [25]:
data = data[data["movieId"].isin(usersPerMOVIE[usersPerMOVIE>10].index)]

In [26]:
data = data[data["userId"].isin(MOVIESsPerUser[MOVIESsPerUser>10].index)]

In [27]:
userItemRatingMatrix=pd.pivot_table(data, values='rating',
                                    index=['userId'], columns=['movieId'])

In [28]:
userItemRatingMatrix.head()

movieId,1,2,3,4,5,6,7,9,10,11,...,8957,8958,8961,8966,8970,8972,8981,8983,8984,8985
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
userItemRatingMatrix.shape

(666, 1759)

In [30]:
user1 = 1
user2 = 20

In [31]:
user1Ratings = userItemRatingMatrix.transpose()[user1]
user1Ratings.head(50)

movieId
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
9     NaN
10    NaN
11    NaN
12    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
22    NaN
23    NaN
24    NaN
25    NaN
28    NaN
29    NaN
31    2.5
32    NaN
34    NaN
35    NaN
36    NaN
39    NaN
41    NaN
43    NaN
44    NaN
45    NaN
46    NaN
47    NaN
48    NaN
50    NaN
52    NaN
57    NaN
58    NaN
60    NaN
62    NaN
65    NaN
69    NaN
70    NaN
73    NaN
74    NaN
76    NaN
Name: 1, dtype: float64

In [32]:
user2Ratings = userItemRatingMatrix.transpose()[user2]

In [33]:
from scipy.spatial.distance import hamming 
hamming(user1Ratings,user2Ratings)

1.0

In [34]:
import numpy as np
def distance(user1,user2):
        try:
            user1Ratings = userItemRatingMatrix.transpose()[user1]
            user2Ratings = userItemRatingMatrix.transpose()[user2]
            distance = hamming(user1Ratings,user2Ratings)
        except: 
            distance = np.NaN
        return distance 

In [35]:
distance(6,5)

0.9994314951677089

In [36]:
user = 6
allUsers = pd.DataFrame(userItemRatingMatrix.index)
allUsers = allUsers[allUsers.userId!=user]
allUsers.head(10)

,userId
0,1
1,2
2,3
3,4
4,5
6,7
7,8
8,9
9,10
10,11


In [37]:
allUsers["distance"] = allUsers["userId"].apply(lambda x: distance(user,x))

In [38]:
allUsers.head()

,userId,distance
0,1,1.000000
1,2,1.000000
2,3,1.000000
3,4,1.000000
4,5,0.999431


In [39]:
K = 10
KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["userId"][:K]

In [40]:
KnearestUsers

72      73
465    468
252    253
118    119
29      30
104    105
649    654
544    547
558    561
200    201
Name: userId, dtype: int64

In [41]:
def nearestNeighbors(user,K=10):
    allUsers = pd.DataFrame(userItemRatingMatrix.index)
    allUsers = allUsers[allUsers.userId!=user]
    allUsers["distance"] = allUsers["userId"].apply(lambda x: distance(user,x))
    KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["userId"][:K]
    return KnearestUsers

In [42]:
KnearestUsers = nearestNeighbors(user)

In [43]:
KnearestUsers

72      73
465    468
252    253
118    119
29      30
104    105
649    654
544    547
558    561
200    201
Name: userId, dtype: int64

In [44]:
NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
NNRatings

movieId,1,2,3,4,5,6,7,9,10,11,...,8957,8958,8961,8966,8970,8972,8981,8983,8984,8985
userId,,,,,,,,,,,,,,,,,,,,,
30,4.0,2.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,5.0,2.5,NaN,NaN,NaN,4.5,NaN,NaN,3.0,NaN,...,4.0,NaN,3.0,NaN,NaN,3.0,NaN,NaN,2.5,NaN
105,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.5,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,2.0,3.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,5.0,5.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN
253,4.5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
468,4.0,2.0,2.5,NaN,NaN,3.5,NaN,NaN,3.0,2.0,...,NaN,3.0,3.5,NaN,3.0,NaN,3.0,NaN,NaN,NaN
547,3.5,NaN,NaN,NaN,NaN,2.5,2.0,NaN,NaN,3.0,...,NaN,4.5,NaN,4.0,3.5,NaN,4.0,NaN,2.5,NaN
561,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,...,NaN,NaN,4.5,NaN,NaN,4.0,NaN,NaN,4.0,NaN


In [45]:
avgRating = NNRatings.apply(np.nanmean).dropna()
avgRating.head()

/home/puc/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4933: RuntimeWarning: Mean of empty slice
  labels=labels)


movieId
1    4.000000
2    2.785714
3    2.500000
5    4.000000
6    3.928571
dtype: float64

In [46]:
moviesAlreadyWatched = userItemRatingMatrix.transpose()[user].dropna().index
moviesAlreadyWatched

Int64Index([ 111,  158,  173,  293,  596,  903, 1204, 1250, 1259, 1276, 1285,
            1358, 1639, 1687, 1747, 1876, 1909, 2001, 2019, 2072, 2174, 2502,
            2528, 2529, 2571, 2657, 2692, 2723, 2761, 2890, 3052, 3114, 3300,
            3751, 4641, 4975, 5952, 7090, 7153, 7361, 8368, 8636, 8784, 8874],
           dtype='int64', name='movieId')

In [47]:
avgRating = avgRating[~avgRating.index.isin(moviesAlreadyWatched)]

In [48]:
N=3
topNMOVIEIDs = avgRating.sort_values(ascending=False).index[:N]

In [49]:
pd.Series(topNMOVIEIDs).apply(movieMeta)

0                                (Wilde (1997), Drama)
1    (Amazing Panda Adventure, The (1995), Adventur...
2    (Postman, The (1997), Action|Adventure|Drama|S...
Name: movieId, dtype: object

In [50]:
def topN(user,N=3):
    KnearestUsers = nearestNeighbors(user)
    NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
    avgRating = NNRatings.apply(np.nanmean).dropna()
    moviesAlreadyWatched = userItemRatingMatrix.transpose()[user].dropna().index
    avgRating = avgRating[~avgRating.index.isin(moviesAlreadyWatched)]
    ratingPredictedValue = avgRating.sort_values(ascending=False)
    #print(ratingPredictedValue.head(N))
    topNMOVIEIDs = avgRating.sort_values(ascending=False).index[:N]
    recommendation = pd.DataFrame(topNMOVIEIDs) 
    recommendation["title"] = recommendation["movieId"].apply(movieMeta)
    recommendation["Prediction"] = ratingPredictedValue.values[:N]
    return recommendation

In [51]:
faveMovies(3,10)

,userId,movieId,rating,timestamp,title
133,3,3949,5.0,1298863174,"(Sandlot, The (1993), Children|Comedy|Drama)"
113,3,1197,5.0,1298932770,"(Crash (1996), Drama|Thriller)"
101,3,318,5.0,1298862121,"(Crow, The (1994), Action|Crime|Fantasy|Thriller)"
103,3,356,5.0,1298862167,"(Mirage (1995), Action|Thriller)"
130,3,2959,5.0,1298862874,"(Bird on a Wire (1990), Action|Comedy|Romance)"
100,3,296,4.5,1298862418,"(Tom & Viv (1994), Drama)"
119,3,1721,4.5,1298923236,"(Return to Paradise (1998), Crime|Drama|Romanc..."
97,3,110,4.0,1298922049,"(Boomerang (1992), Comedy|Romance)"
129,3,2858,4.0,1298921825,"(Hidden, The (1987), Action|Horror|Sci-Fi)"
111,3,778,4.0,1298863157,"(Angel on My Shoulder (1946), Crime|Drama)"


In [52]:
topN(3,70)

/home/puc/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4933: RuntimeWarning: Mean of empty slice
  labels=labels)


,movieId,title,Prediction
0,3524,"(Adventures of Baron Munchausen, The (1988), A...",5.000000
1,1588,"(Disturbing Behavior (1998), Horror|Thriller)",5.000000
2,913,"(Hustler White (1996), Romance)",5.000000
3,1147,"(Whole Wide World, The (1996), Drama)",5.000000
4,3468,"(Crimson Rivers, The (Rivières pourpres, Les) ...",5.000000
5,3359,"(River's Edge (1986), Crime|Drama)",5.000000
6,3469,"(Lumumba (2000), Drama)",5.000000
7,273,"(Ready to Wear (Pret-A-Porter) (1994), Comedy)",5.000000
8,1095,"(Blood and Wine (Blood & Wine) (1996), Crime|D...",5.000000
9,599,"(Truth About Cats & Dogs, The (1996), Comedy|R...",5.000000


In [53]:
from pymongo import MongoClient

#Conexão com o MongoDB
client = MongoClient() 
db = client['movies']
db_recomendacoes = db['knn_recomendacoes']

In [54]:
#Número de recomendações desejadas
nrec = 10
#Recupera lista de usuários a partir da matriz de avaliações
usuarios = pd.DataFrame(userItemRatingMatrix.index)

In [45]:
for uid in usuarios["userId"]:
#topN retorna os N filmes mais recomendados com base no algortimo de vizinhos mais pŕoximos
    recs = topN(uid,nrec)
    
#Cria o documento contendo as recomendações para o usuário
    rec_doc = {
        "userId": uid,
        "recs": recs["title"].values.tolist(),
        "prediction": recs["Prediction"].values.tolist()
    }
    
#Insere as recomendações do usuário no MongoDB
    db_recomendacoes.insert_one(rec_doc)
 



/home/puc/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4933: RuntimeWarning: Mean of empty slice
  labels=labels)
